In [1]:
import os, shutil

# 存放小数据集的目录
base_dir = '/Users/fanhaoran/MyDeepLearning/2019.5/catVSdog/dogs-vs-cats'

# 建立训练集、验证集、测试集目录
train_dir = os.path.join(base_dir, 'train')
# os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
# os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
# os.mkdir(test_dir)

# 将猫狗照片按照训练、验证、测试分类
train_cats_dir = os.path.join(train_dir, 'cat')
#os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dog')
#os.mkdir(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cat')
#os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dog')
#os.mkdir(validation_dogs_dir)
test_cats_dir = os.path.join(test_dir, 'cat')
#os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dog')
#os.mkdir(test_dogs_dir)
print('total train_cat_img is ', len(os.listdir(train_cats_dir)))
print('total train_dog_img is ', len(os.listdir(train_dogs_dir)))
print('total validation_cat_img is ', len(os.listdir(validation_cats_dir)))
print('total validation_dog_img is ', len(os.listdir(validation_dogs_dir)))
print('total test_cat_img is ', len(os.listdir(test_cats_dir)))
print('total test_dog_img is ', len(os.listdir(test_dogs_dir)))

total train_cat_img is  802
total train_dog_img is  801
total validation_cat_img is  101
total validation_dog_img is  100
total test_cat_img is  101
total test_dog_img is  100


In [2]:
#检查读入的图片
'''import cv2
index = 'cat.1.jpg'
img_address = os.path.join(train_cats_dir,index)
img = cv2.imread(img_address,1)
cv2.imshow('show.jpg',img)
cv2.waitKey(0)'''

"import cv2\nindex = 'cat.1.jpg'\nimg_address = os.path.join(train_cats_dir,index)\nimg = cv2.imread(img_address,1)\ncv2.imshow('show.jpg',img)\ncv2.waitKey(0)"

In [3]:
import tensorflow as tf
import numpy as np
from keras import layers
from keras import models

# 搭建一个模型
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [4]:
from keras import optimizers

#配置模型
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# 使用 ImageDataGenerator 从目录中读取图像
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255) # 将所有图像乘以 1/255 缩放
train_generator = train_datagen.flow_from_directory( 
        train_dir, #目标目录
        target_size=(150, 150),  #将所有图像的大小调整为 150×150
        batch_size=20, 
        class_mode='binary') # 因为使用了 binary_crossentropy 损失,所以需要用二进制标签
validation_generator = test_datagen.flow_from_directory( 
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

# 利用批量生成器拟合模型
history = model.fit_generator(
    train_generator,
    steps_per_epoch=40,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=10)


Found 1602 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/20
40/40 [==============================] - 19s 470ms/step - loss: 0.6937 - acc: 0.5112 - val_loss: 0.6917 - val_acc: 0.5000
Epoch 2/20
40/40 [==============================] - 17s 433ms/step - loss: 0.6882 - acc: 0.5675 - val_loss: 0.6787 - val_acc: 0.5400
Epoch 3/20
40/40 [==============================] - 17s 434ms/step - loss: 0.6675 - acc: 0.5941 - val_loss: 0.6735 - val_acc: 0.5300
Epoch 4/20
40/40 [==============================] - 16s 412ms/step - loss: 0.6699 - acc: 0.5813 - val_loss: 0.6549 - val_acc: 0.5900
Epoch 5/20
40/40 [==============================] - 16s 397ms/step - loss: 0.6403 - acc: 0.6379 - val_loss: 0.6527 - val_acc: 0.6100
Epoch 6/20
40/40 [==============================] - 15s 386ms/step - loss: 0.6397 - acc: 0.6412 - val_loss: 0.6558 - val_acc: 0.5500
Epoch 7/20
40/40 [==============================] - 16s 395ms/step - loss: 0.5986 - acc: 0.6748 - val_loss: 0.6260 - va

In [5]:
#save
model.save('1st.h5')
#delete existing model
#del model

from keras.models import load_model
#load 
model = load_model('1st.h5')


NameError: name 'load_model' is not defined

In [ ]:
"""Found 1602 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/20
40/40 [==============================] - 17s 434ms/step - loss: 0.6902 - acc: 0.5013 - val_loss: 0.6823 - val_acc: 0.5400
Epoch 2/20
40/40 [==============================] - 14s 360ms/step - loss: 0.6805 - acc: 0.5588 - val_loss: 0.6777 - val_acc: 0.5150
Epoch 3/20
40/40 [==============================] - 17s 415ms/step - loss: 0.6545 - acc: 0.5853 - val_loss: 0.6607 - val_acc: 0.5800
Epoch 4/20
40/40 [==============================] - 17s 427ms/step - loss: 0.6474 - acc: 0.6175 - val_loss: 0.6568 - val_acc: 0.5850
Epoch 5/20
40/40 [==============================] - 17s 422ms/step - loss: 0.6330 - acc: 0.6513 - val_loss: 0.6382 - val_acc: 0.6250
Epoch 6/20
40/40 [==============================] - 18s 452ms/step - loss: 0.5987 - acc: 0.6850 - val_loss: 0.6137 - val_acc: 0.6450
Epoch 7/20
40/40 [==============================] - 17s 429ms/step - loss: 0.5643 - acc: 0.7101 - val_loss: 0.6042 - val_acc: 0.6450
Epoch 8/20
40/40 [==============================] - 16s 400ms/step - loss: 0.5814 - acc: 0.6825 - val_loss: 0.6126 - val_acc: 0.6750
Epoch 9/20
40/40 [==============================] - 15s 380ms/step - loss: 0.5654 - acc: 0.7211 - val_loss: 0.6160 - val_acc: 0.6600
Epoch 10/20
40/40 [==============================] - 16s 401ms/step - loss: 0.5425 - acc: 0.7250 - val_loss: 0.6080 - val_acc: 0.6850
Epoch 11/20
40/40 [==============================] - 16s 406ms/step - loss: 0.5431 - acc: 0.7226 - val_loss: 0.5754 - val_acc: 0.6950
Epoch 12/20
40/40 [==============================] - 16s 406ms/step - loss: 0.5131 - acc: 0.7487 - val_loss: 0.6129 - val_acc: 0.6750
Epoch 13/20
40/40 [==============================] - 16s 411ms/step - loss: 0.5112 - acc: 0.7376 - val_loss: 0.5680 - val_acc: 0.6850
Epoch 14/20
40/40 [==============================] - 16s 389ms/step - loss: 0.5157 - acc: 0.7587 - val_loss: 0.5706 - val_acc: 0.7150
Epoch 15/20
40/40 [==============================] - 15s 378ms/step - loss: 0.4933 - acc: 0.7674 - val_loss: 0.6219 - val_acc: 0.6700
Epoch 16/20
40/40 [==============================] - 17s 416ms/step - loss: 0.4869 - acc: 0.7675 - val_loss: 0.5621 - val_acc: 0.7000
Epoch 17/20
40/40 [==============================] - 16s 408ms/step - loss: 0.4844 - acc: 0.7576 - val_loss: 0.5514 - val_acc: 0.7150
Epoch 18/20
40/40 [==============================] - 16s 395ms/step - loss: 0.4588 - acc: 0.7900 - val_loss: 0.5674 - val_acc: 0.6800
Epoch 19/20
40/40 [==============================] - 16s 398ms/step - loss: 0.4558 - acc: 0.7949 - val_loss: 0.5506 - val_acc: 0.7300
Epoch 20/20
40/40 [==============================] - 16s 403ms/step - loss: 0.4354 - acc: 0.7988 - val_loss: 0.5400 - val_acc: 0.7050"""